In [1]:
from __future__ import print_function

from collections import defaultdict

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline


import flotilla
sns.set(style='ticks', context='talk')

figure_folder = 'figures'
! mkdir $figure_folder

flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects/'

study = flotilla.embark('singlecell_pnm_miso_modalities', flotilla_dir=flotilla_dir)

mkdir: cannot create directory `figures': File exists
2016-07-14 13:17:57	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_miso_modalities/datapackage.json
2016-07-14 13:17:57	Parsing datapackage to create a Study object
2016-07-14 13:18:41 	Initializing Study
2016-07-14 13:18:41 	Initializing Predictor configuration manager for Study
2016-07-14 13:18:41	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2016-07-14 13:18:41	Added ExtraTreesClassifier to default predictors
2016-07-14 13:18:41	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2016-07-14 13:18:41	Added ExtraTreesRegressor to default predictors
2016-07-14 13:18:41	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
2016-07-14 13:18:41	Added GradientBoostingClassifier to default predictors
2016-07-14 13:18:41	Predictor GradientB

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [2]:
study.splicing.minimum_samples

10

In [3]:
not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

psi = study.splicing.singles.ix[not_outliers]
grouped = psi.groupby(study.sample_id_to_phenotype)
psi_filtered = grouped.apply(lambda x: x.dropna(axis=1, thresh=study.splicing.minimum_samples))

In [4]:
from astrolabe import Waypoints

ws = Waypoints()

waypoints = psi_filtered.groupby(study.sample_id_to_phenotype).apply(
    lambda x: ws.fit_transform(x.dropna(how='all', axis=1)))

In [5]:
transitions = study.phenotype_transitions + [('iPSC', 'MN')]
transitions

[(u'iPSC', u'NPC'), (u'NPC', u'MN'), ('iPSC', 'MN')]

In [9]:
from astrolabe.voyages import Voyages

v = Voyages()

voyages = v.voyages(waypoints, transitions)
voyages.head()

,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition
0,chr10:101507014:101507147:+@chr10:101510126:10...,0.221198,-0.002304,0.221210,iPSC,NPC,$\searrow$,iPSC-NPC
1,chr10:101945271:101945394:-@chr10:101943513:10...,0.000000,-0.006912,0.006912,iPSC,NPC,$\swarrow$,iPSC-NPC
2,chr10:102013178:102013296:-@chr10:102010005:10...,0.000000,-0.191729,0.191729,iPSC,NPC,$\swarrow$,iPSC-NPC
3,chr10:102019975:102020076:-@chr10:102016019:10...,0.000000,0.189247,0.189247,iPSC,NPC,$\nwarrow$,iPSC-NPC
4,chr10:102020723:102020801:-@chr10:102016019:10...,0.000000,0.002299,0.002299,iPSC,NPC,$\nwarrow$,iPSC-NPC


## Add group1/group2 modalities

In [10]:

groups = 'group1', 'group2'
voyages_modalities = voyages.copy()

for group in groups:
    voyages_modalities = voyages_modalities.merge(study.supplemental.modalities_tidy, 
                                       left_on=[group, 'event_id'], 
                                       right_on=['phenotype', 'event_id'], copy=False)
    print(voyages_modalities.shape)
    voyages_modalities = voyages_modalities.drop(['phenotype'], axis=1)
    voyages_modalities = voyages_modalities.rename(columns={'modality': '{}_modality'.format(group)})
print(voyages_modalities.shape)
voyages_modalities.head()

(27602, 10)
(27602, 11)
(27602, 10)


,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition,group1_modality,group2_modality
0,chr10:101507014:101507147:+@chr10:101510126:10...,0.221198,-0.002304,0.221210,iPSC,NPC,$\searrow$,iPSC-NPC,uncategorized,uncategorized
1,chr10:101945271:101945394:-@chr10:101943513:10...,0.000000,-0.006912,0.006912,iPSC,NPC,$\swarrow$,iPSC-NPC,included,included
2,chr10:102013178:102013296:-@chr10:102010005:10...,0.000000,-0.191729,0.191729,iPSC,NPC,$\swarrow$,iPSC-NPC,included,included
3,chr10:102019975:102020076:-@chr10:102016019:10...,0.000000,0.189247,0.189247,iPSC,NPC,$\nwarrow$,iPSC-NPC,included,included
4,chr10:102020723:102020801:-@chr10:102016019:10...,0.000000,0.002299,0.002299,iPSC,NPC,$\nwarrow$,iPSC-NPC,included,included


## Add data to the study and save it

In [11]:

study.supplemental.waypoints = waypoints
study.supplemental.voyages = voyages_modalities

In [12]:
study.save('singlecell_pnm_miso_modalities_voyages', flotilla_dir=flotilla_dir)

Wrote datapackage to /projects/ps-yeolab/obotvinnik/flotilla_projects//singlecell_pnm_miso_modalities_voyages/datapackage.json
